## Importing the necessary dependencies

In [21]:
!pip install requests beautifulsoup4 selenium pandas

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/2f/a6/fc66ea71ec0769f72abdf15cb9ec9269517abe68a160839383ddff7478f1/selenium-4.29.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/c9/55/c4d9bea8b3d7937901958f65124123512419ab0eb73695e5f382521abbfb/trio-0.29.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/c7/19/eb640a397bba49ba49ef9dbe2e7e5c04202ba045b6ce2ec36e9cadc51e04/trio_websocket-0.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for websocket-client~=1.8 from https://files.pythonhosted.org/packages/5a/84/44687a29792a70e111c5c477230a72c4b957d88d16141199bf9acb7537a3/websocket_client-1.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for outcome from https://files.pythonhosted.org/packages/55/8b/5ab7257531a5d830fc8000c476e63c935488d74609b50f9384a643ec0a6

In [56]:
!pip install undetected_chromedriver

     ---------------------------------------- 0.0/65.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/65.4 kB ? eta -:--:--
     ----------------------- -------------- 41.0/65.4 kB 487.6 kB/s eta 0:00:01
     -------------------------------------- 65.4/65.4 kB 707.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for websockets from https://files.pythonhosted.org/packages/98/93/e36c73f78400a65f5e236cd376713c34182e6663f6889cd45a4a04d8f203/websockets-15.0.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/176.8 kB ? eta -:--:--
   ----------------------- ---------------- 102.4/176.8 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 176.8/176.8 kB 3.5 MB/s eta 0:00:00
  Created wheel for undetected_chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47130 sha256=81d95b0a4b690c68e8c3342a86b2

In [1]:
import pandas as pd
import requests

import undetected_chromedriver as uc
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

C:\Users\nawaf\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Predefined Functions

In [43]:
# Convert "X / Y" to percentage
def to_percentage(value):
    if (value == '-'):
        return f"{(0 / 1) :.2f}"
    x, y = map(int, value.split(' / '))
    return f"{(x / y) :.2f}"

# Mapping function
def get_advantage(xvy):
    x, y = map(int, xvy.split('v'))
    return f"+{x - y}" if x - y > 0 else f"{x - y}"

## Scraping the Data

#### Pistol Rounds, First Kills, Clutch%

In [2]:
driver = uc.Chrome()
driver.get("https://www.rib.gg/series/100-thieves-vs-nrg-esports-champions-tour-2025-americas-kickoff/83724?tab=team-stats")

print("Waiting for page to load...")
time.sleep(10)  # Give the page time to fully load

# Extract the team names
team_elements = driver.find_elements(By.XPATH, "//p[contains(@class, 'css-1aujtoy') and string-length(text()) > 0]")
teams = [team.text.strip() for team in team_elements if len(team.text.strip()) > 0][:2]  # Extract first two team names

print(f"Teams detected: {teams}")

result_elements = driver.find_elements(By.XPATH, "//p[contains(@class, 'css-tesx4r') or contains(@class, 'css-hov9j3') and string-length(text()) > 0]")
results = [result.text.strip() for result in result_elements if len(result.text.strip()) > 0][:2]  # Extract the win and loss results

print(f"Results detected: {results}")

# Extract the statistics
all_stats = {}

# Find all stat categories
stat_names = driver.find_elements(By.XPATH, "//div[contains(text(), 'Pistol Rounds Won') or contains(text(), 'First Kills') or contains(text(), 'KAST') or contains(text(), 'Clutches')]")

for stat in stat_names:
    stat_name = stat.text
    print(f"\nFound stat: {stat_name}")

    try:
        # Look for numerical values in adjacent elements
        team1_value_elements = stat.find_elements(By.XPATH, "./preceding-sibling::div[contains(@class, 'css-') and string-length(text()) > 0]")

#         team1_values = [el.text.strip() for el in team1_value_elements if el.text.strip().replace('%', '').replace('.', '').isdigit()]  # Ensure values are numbers

        
        team2_value_elements = stat.find_elements(By.XPATH, "./following-sibling::div[contains(@class, 'css-') and string-length(text()) > 0]")

#         team2_values = [el.text.strip() for el in team2_value_elements if el.text.strip().replace('%', '').replace('.', '').isdigit()]  # Ensure values are numbers
        
        
        print(f"  {teams[0]} {stat_name}: {team1_value_elements[0].text.strip().replace('%', '').replace('.', '')}")
        print(f"  {teams[1]} {stat_name}: {team2_value_elements[0].text.strip().replace('%', '').replace('.', '')}")
#         if len(values) == 2:  # We expect exactly two values (one per team)
#             all_stats[stat_name] = values
#         else:
#             print(f"  Skipped due to unexpected format: {values}")

        all_stats[stat_name] = [team1_value_elements[0].text.strip().replace('%', '').replace('.', ''), team2_value_elements[0].text.strip().replace('%', '').replace('.', '')]
    
    except Exception as e:
        print(f"Error extracting {stat_name}: {e}")

# Create a DataFrame
if all_stats and len(teams) == 2:
    main_df = pd.DataFrame.from_dict(all_stats, orient='index', columns=teams).transpose()
    print("\nExtracted Data:")
    print(main_df)

    # Converting the clutches into percentages
    main_df["Clutches"] = main_df["Clutches"].str.split(" / ").apply(lambda x: int(x[0]) / int(x[1]) if int(x[1]) != 0 else 0)
    main_df['KAST'] = main_df['KAST'].astype('float').div(100)
    
    main_df = main_df.astype('float')
    
    # Save to CSV
#     df.to_csv("team_stats.csv")
#     print("Data saved to team_stats.csv")
else:
    print("Failed to extract meaningful stat data")

driver.quit()

Waiting for page to load...
Teams detected: ['100 Thieves', 'NRG Esports']
Teams detected: ['Loser', 'Winner']

Found stat: Pistol Rounds Won
  100 Thieves Pistol Rounds Won: 2
  NRG Esports Pistol Rounds Won: 4

Found stat: First Kills
  100 Thieves First Kills: 36
  NRG Esports First Kills: 35

Found stat: KAST
  100 Thieves KAST: 72
  NRG Esports KAST: 78

Found stat: Clutches
  100 Thieves Clutches: 9 / 42
  NRG Esports Clutches: 2 / 30

Extracted Data:
            Pistol Rounds Won First Kills KAST Clutches
100 Thieves                 2          36   72   9 / 42
NRG Esports                 4          35   78   2 / 30


In [4]:
main_df

,Pistol Rounds Won,First Kills,KAST,Clutches
100 Thieves,2,36,0.72,0.214286
NRG Esports,4,35,0.78,0.066667


#### Economy Data

In [5]:
driver = uc.Chrome()
driver.get("https://www.rib.gg/series/100-thieves-vs-nrg-esports-champions-tour-2025-americas-kickoff/83724?tab=team-stats")

print("Waiting for page to load...")
time.sleep(10)  # Give the page time to fully load

# Extract the team names
team_elements = driver.find_elements(By.XPATH, "//p[contains(@class, 'css-1aujtoy') and string-length(text()) > 0]")
teams = [team.text.strip() for team in team_elements if len(team.text.strip()) > 0][:2]  # Extract first two team names

print(f"Teams detected: {teams}")

result_elements = driver.find_elements(By.XPATH, "//p[contains(@class, 'css-tesx4r') or contains(@class, 'css-hov9j3') and string-length(text()) > 0]")
results = [result.text.strip() for result in result_elements if len(result.text.strip()) > 0][:2]  # Extract the win and loss results

print(f"Results detected: {results}")

# Extract the statistics
all_stats = {}

# Find all stat categories
stat_names = driver.find_elements(By.XPATH, "//div[text()='Eco' or text()='Semi-Eco' or text()='Half-Buy' or text()='Full-Buy']")

for stat in stat_names:
    stat_name = stat.text
    print(f"\nFound stat: {stat_name}")

    try:
        # Look for numerical values in adjacent elements
        team1_value_elements = stat.find_elements(By.XPATH, "./parent::div/preceding-sibling::div/child::div[contains(@class, 'css-') and string-length(text()) > 0]")

#         team1_values = [el.text.strip() for el in team1_value_elements if el.text.strip().replace('%', '').replace('.', '').isdigit()]  # Ensure values are numbers

        
        team2_value_elements = stat.find_elements(By.XPATH, "./parent::div/following-sibling::div/child::div[contains(@class, 'css-') and string-length(text()) > 0]")

#         team2_values = [el.text.strip() for el in team2_value_elements if el.text.strip().replace('%', '').replace('.', '').isdigit()]  # Ensure values are numbers
        
        
        print(f"  {teams[0]} {stat_name}: {team1_value_elements[0].text.strip()}")
        print(f"  {teams[1]} {stat_name}: {team2_value_elements[0].text.strip()}")
#         if len(values) == 2:  # We expect exactly two values (one per team)
#             all_stats[stat_name] = values
#         else:
#             print(f"  Skipped due to unexpected format: {values}")

        all_stats[stat_name] = [team1_value_elements[0].text.strip().replace('%', '').replace('.', ''), team2_value_elements[0].text.strip().replace('%', '').replace('.', '')]
    
    except Exception as e:
        print(f"Error extracting {stat_name}: {e}")

# Create a DataFrame
if all_stats and len(teams) == 2:
    eco_df = pd.DataFrame.from_dict(all_stats, orient='index', columns=teams).transpose()
    print("\nExtracted Data:")
#     print(df)

    eco_df = eco_df.astype('float').div(100) # Transpose and divide by 100 to turn percentages into decimals

    # Save to CSV
#     df.to_csv("team_stats.csv")
#     print("Data saved to team_stats.csv")
else:
    print("Failed to extract meaningful stat data")

driver.quit()

Waiting for page to load...
Teams detected: ['100 Thieves', 'NRG Esports']
Teams detected: ['Loser', 'Winner']

Found stat: Eco
  100 Thieves Eco: 0%
  NRG Esports Eco: 0%

Found stat: Semi-Eco
  100 Thieves Semi-Eco: 0%
  NRG Esports Semi-Eco: 60%

Found stat: Half-Buy
  100 Thieves Half-Buy: 54%
  NRG Esports Half-Buy: 77%

Found stat: Full-Buy
  100 Thieves Full-Buy: 51%
  NRG Esports Full-Buy: 48%

Extracted Data:


In [6]:
eco_df

,Eco,Semi-Eco,Half-Buy,Full-Buy
100 Thieves,0.0,0.0,0.54,0.51
NRG Esports,0.0,0.6,0.77,0.48


#### XvY Conversions

In [26]:
driver = uc.Chrome()
driver.get("https://www.rib.gg/series/100-thieves-vs-nrg-esports-champions-tour-2025-americas-kickoff/83724?tab=team-stats")

print("Waiting for page to load...")
time.sleep(10)  # Give the page time to fully load

# Extract the team names
team_elements = driver.find_elements(By.XPATH, "//p[contains(@class, 'css-1aujtoy') and string-length(text()) > 0]")
teams = [team.text.strip() for team in team_elements if len(team.text.strip()) > 0][:2]  # Extract first two team names

print(f"Teams detected: {teams}")

result_elements = driver.find_elements(By.XPATH, "//p[contains(@class, 'css-tesx4r') or contains(@class, 'css-hov9j3') and string-length(text()) > 0]")
results = [result.text.strip() for result in result_elements if len(result.text.strip()) > 0][:2]  # Extract the win and loss results

print(f"Results detected: {results}")

# Extract the statistics
all_stats = {}

# Find all stat categories
stat_names = driver.find_elements(By.XPATH, "//div[contains(text(), '5v4') or contains(text(), '4v5') or contains(text(), '4v4') or contains(text(), '4v3') or contains(text(), '3v4') or contains(text(), '3v3') or contains(text(), '5v3') or contains(text(), '3v5') or contains(text(), '3v2') or contains(text(), '2v3') or contains(text(), '4v2') or contains(text(), '2v4') or contains(text(), '2v2') or contains(text(), '3v1') or contains(text(), '1v3') or contains(text(), '1v1') or contains(text(), '2v1') or contains(text(), '1v2') or contains(text(), '1v4') or contains(text(), '4v1') or contains(text(), '5v2') or contains(text(), '2v5') or contains(text(), '5v1') or contains(text(), '1v5')]")

for stat in stat_names:
    stat_name = stat.text
    print(f"\nFound stat: {stat_name}")

    try:
        # Look for numerical values in adjacent elements
        team1_value_elements = stat.find_elements(By.XPATH, "./parent::div/parent::div/preceding-sibling::div[contains(@class, 'css-') and string-length(text()) > 0]")

#         team1_values = [el.text.strip() for el in team1_value_elements if el.text.strip().replace('%', '').replace('.', '').isdigit()]  # Ensure values are numbers

        
        team2_value_elements = stat.find_elements(By.XPATH, "./parent::div/parent::div/following-sibling::div[contains(@class, 'css-') and string-length(text()) > 0]")

#         team2_values = [el.text.strip() for el in team2_value_elements if el.text.strip().replace('%', '').replace('.', '').isdigit()]  # Ensure values are numbers
        
        
        print(f"  {teams[0]} {stat_name}: {team1_value_elements[0].text.strip()}")
        print(f"  {teams[1]} {stat_name}: {team2_value_elements[0].text.strip()}")
#         if len(values) == 2:  # We expect exactly two values (one per team)
#             all_stats[stat_name] = values
#         else:
#             print(f"  Skipped due to unexpected format: {values}")

        all_stats[stat_name] = [team1_value_elements[0].text.strip().replace('%', '').replace('.', ''), team2_value_elements[0].text.strip().replace('%', '').replace('.', '')]
    
    except Exception as e:
        print(f"Error extracting {stat_name}: {e}")

# Create a DataFrame
if all_stats and len(teams) == 2:
    xvy_df = pd.DataFrame.from_dict(all_stats, orient='index', columns=teams).transpose()
    print("\nExtracted Data:")
#     print(df)

    # Apply transformation to all columns except 'Team'
    for col in xvy_df.columns:  
        xvy_df[col] = xvy_df[col].apply(to_percentage)

    xvy_df = xvy_df.astype('float')
    
    # Create new aggregated DataFrame
    agg_data = {}
    counts = {}

    for col in xvy_df.columns:
        category = get_advantage(col)
        if category not in agg_data:
            agg_data[category] = xvy_df[col].copy()
            counts[category] = 1
        else:
            agg_data[category] += xvy_df[col]
            counts[category] += 1

    # Convert to DataFrame and take the average
    xvy_df = pd.DataFrame({key: agg_data[key] / counts[key] for key in agg_data})

    # Save to CSV
#     df.to_csv("team_stats.csv")
#     print("Data saved to team_stats.csv")
else:
    print("Failed to extract meaningful stat data")

driver.quit()

Waiting for page to load...
Teams detected: ['100 Thieves', 'NRG Esports']
Teams detected: ['Loser', 'Winner']

Found stat: 5v4
  100 Thieves 5v4: 8 / 12
  NRG Esports 5v4: 12 / 18

Found stat: 4v5
  100 Thieves 4v5: 6 / 18
  NRG Esports 4v5: 4 / 12

Found stat: 4v4
  100 Thieves 4v4: 9 / 21
  NRG Esports 4v4: 12 / 21

Found stat: 4v3
  100 Thieves 4v3: 4 / 7
  NRG Esports 4v3: 9 / 13

Found stat: 3v4
  100 Thieves 3v4: 4 / 13
  NRG Esports 3v4: 3 / 7

Found stat: 3v3
  100 Thieves 3v3: 6 / 10
  NRG Esports 3v3: 4 / 10

Found stat: 5v3
  100 Thieves 5v3: 3 / 4
  NRG Esports 5v3: 6 / 8

Found stat: 3v5
  100 Thieves 3v5: 2 / 8
  NRG Esports 3v5: 1 / 4

Found stat: 3v2
  100 Thieves 3v2: 3 / 3
  NRG Esports 3v2: 4 / 7

Found stat: 2v3
  100 Thieves 2v3: 3 / 7
  NRG Esports 2v3: 0 / 3

Found stat: 4v2
  100 Thieves 4v2: 5 / 5
  NRG Esports 4v2: 6 / 6

Found stat: 2v4
  100 Thieves 2v4: 0 / 6
  NRG Esports 2v4: 0 / 5

Found stat: 2v2
  100 Thieves 2v2: 5 / 8
  NRG Esports 2v2: 3 / 8

Found

In [27]:
xvy_df

,+1,-1,0,+2,-2,-3,+3,+4,-4
100 Thieves,0.8100,0.3575,0.6125,0.916667,0.083333,0.0,1.0,1.0,0.0
NRG Esports,0.6425,0.1900,0.3875,0.916667,0.083333,0.0,1.0,1.0,0.0


In [26]:
agg_df

,+1,-1,0,+2,-2,-3,+3,+4,-4
100 Thieves,0.8100,0.3575,0.6125,0.916667,0.083333,0.0,1.0,1.0,0.0
NRG Esports,0.6425,0.1900,0.3875,0.916667,0.083333,0.0,1.0,1.0,0.0


---
### Aggregating all the data into one df

In [82]:
url = "https://www.rib.gg/series/83355?tab=team-stats"

In [37]:
driver = uc.Chrome()
driver.get(url)

print("Waiting for page to load...")
time.sleep(10)  # Give the page time to fully load

# Extract the team names
team_elements = driver.find_elements(By.XPATH, "//p[contains(@class, 'css-1aujtoy') and string-length(text()) > 0]")
teams = [team.text.strip() for team in team_elements if len(team.text.strip()) > 0][:2]  # Extract first two team names

print(f"Teams detected: {teams}")

result_elements = driver.find_elements(By.XPATH, "//p[contains(@class, 'css-tesx4r') or contains(@class, 'css-hov9j3') and string-length(text()) > 0]")
results = [result.text.strip() for result in result_elements if len(result.text.strip()) > 0][:2]  # Extract the win and loss results

print(f"Results detected: {results}")


# PISTOL ROUNDS, FIRST KILLS, KAST #

# Extract the statistics
all_stats = {}

# Find all stat categories
stat_names = driver.find_elements(By.XPATH, "//div[contains(text(), 'Pistol Rounds Won') or contains(text(), 'First Kills') or contains(text(), 'KAST') or contains(text(), 'Clutches')]")

for stat in stat_names:
    stat_name = stat.text
#     print(f"\nFound stat: {stat_name}")

    try:
        # Look for numerical values in adjacent elements
        team1_value_elements = stat.find_elements(By.XPATH, "./preceding-sibling::div[contains(@class, 'css-') and string-length(text()) > 0]")

#         team1_values = [el.text.strip() for el in team1_value_elements if el.text.strip().replace('%', '').replace('.', '').isdigit()]  # Ensure values are numbers

        
        team2_value_elements = stat.find_elements(By.XPATH, "./following-sibling::div[contains(@class, 'css-') and string-length(text()) > 0]")

#         team2_values = [el.text.strip() for el in team2_value_elements if el.text.strip().replace('%', '').replace('.', '').isdigit()]  # Ensure values are numbers
        
        
#         print(f"  {teams[0]} {stat_name}: {team1_value_elements[0].text.strip().replace('%', '').replace('.', '')}")
#         print(f"  {teams[1]} {stat_name}: {team2_value_elements[0].text.strip().replace('%', '').replace('.', '')}")
#         if len(values) == 2:  # We expect exactly two values (one per team)
#             all_stats[stat_name] = values
#         else:
#             print(f"  Skipped due to unexpected format: {values}")

        all_stats[stat_name] = [team1_value_elements[0].text.strip().replace('%', '').replace('.', ''), team2_value_elements[0].text.strip().replace('%', '').replace('.', '')]
    
    except Exception as e:
        print(f"Error extracting {stat_name}: {e}")

# Create a DataFrame
if all_stats and len(teams) == 2:
    main_df = pd.DataFrame.from_dict(all_stats, orient='index', columns=teams).transpose()
#     print("\nExtracted Data:")
    print(main_df)

    # Converting the clutches into percentages
    main_df["Clutches"] = main_df["Clutches"].str.split(" / ").apply(lambda x: int(x[0]) / int(x[1]) if int(x[1]) != 0 else 0)
    main_df['KAST'] = main_df['KAST'].astype('float').div(100)
    
    # Save to CSV
#     df.to_csv("team_stats.csv")
#     print("Data saved to team_stats.csv")
else:
    print("Failed to extract meaningful stat data")
    
    
# ECO INFORMATION #

# Extract the statistics
all_stats = {}

# Find all stat categories
stat_names = driver.find_elements(By.XPATH, "//div[text()='Eco' or text()='Semi-Eco' or text()='Half-Buy' or text()='Full-Buy']")

for stat in stat_names:
    stat_name = stat.text
#     print(f"\nFound stat: {stat_name}")

    try:
        # Look for numerical values in adjacent elements
        team1_value_elements = stat.find_elements(By.XPATH, "./parent::div/preceding-sibling::div/child::div[contains(@class, 'css-') and string-length(text()) > 0]")

#         team1_values = [el.text.strip() for el in team1_value_elements if el.text.strip().replace('%', '').replace('.', '').isdigit()]  # Ensure values are numbers

        
        team2_value_elements = stat.find_elements(By.XPATH, "./parent::div/following-sibling::div/child::div[contains(@class, 'css-') and string-length(text()) > 0]")

#         team2_values = [el.text.strip() for el in team2_value_elements if el.text.strip().replace('%', '').replace('.', '').isdigit()]  # Ensure values are numbers
        
        
#         print(f"  {teams[0]} {stat_name}: {team1_value_elements[0].text.strip()}")
#         print(f"  {teams[1]} {stat_name}: {team2_value_elements[0].text.strip()}")
#         if len(values) == 2:  # We expect exactly two values (one per team)
#             all_stats[stat_name] = values
#         else:
#             print(f"  Skipped due to unexpected format: {values}")

        all_stats[stat_name] = [team1_value_elements[0].text.strip().replace('%', '').replace('.', '').replace('-', '0'), team2_value_elements[0].text.strip().replace('%', '').replace('.', '').replace('-', '0')]
    
    except Exception as e:
        print(f"Error extracting {stat_name}: {e}")

# Create a DataFrame
if all_stats and len(teams) == 2:
    eco_df = pd.DataFrame.from_dict(all_stats, orient='index', columns=teams).transpose()
#     print("\nExtracted Data:")
#     print(df)

    eco_df = eco_df.astype('float').div(100) # Transpose and divide by 100 to turn percentages into decimals

    # Save to CSV
#     df.to_csv("team_stats.csv")
#     print("Data saved to team_stats.csv")
else:
    print("Failed to extract meaningful stat data")
    
# XVY CONVERSIONS #
    
# Extract the statistics
all_stats = {}

# Find all stat categories
stat_names = driver.find_elements(By.XPATH, "//div[contains(text(), '5v4') or contains(text(), '4v5') or contains(text(), '4v4') or contains(text(), '4v3') or contains(text(), '3v4') or contains(text(), '3v3') or contains(text(), '5v3') or contains(text(), '3v5') or contains(text(), '3v2') or contains(text(), '2v3') or contains(text(), '4v2') or contains(text(), '2v4') or contains(text(), '2v2') or contains(text(), '3v1') or contains(text(), '1v3') or contains(text(), '1v1') or contains(text(), '2v1') or contains(text(), '1v2') or contains(text(), '1v4') or contains(text(), '4v1') or contains(text(), '5v2') or contains(text(), '2v5') or contains(text(), '5v1') or contains(text(), '1v5')]")

for stat in stat_names:
    stat_name = stat.text
#     print(f"\nFound stat: {stat_name}")

    try:
        # Look for numerical values in adjacent elements
        team1_value_elements = stat.find_elements(By.XPATH, "./parent::div/parent::div/preceding-sibling::div[contains(@class, 'css-') and string-length(text()) > 0]")

#         team1_values = [el.text.strip() for el in team1_value_elements if el.text.strip().replace('%', '').replace('.', '').isdigit()]  # Ensure values are numbers

        
        team2_value_elements = stat.find_elements(By.XPATH, "./parent::div/parent::div/following-sibling::div[contains(@class, 'css-') and string-length(text()) > 0]")

#         team2_values = [el.text.strip() for el in team2_value_elements if el.text.strip().replace('%', '').replace('.', '').isdigit()]  # Ensure values are numbers
        
        
#         print(f"  {teams[0]} {stat_name}: {team1_value_elements[0].text.strip()}")
#         print(f"  {teams[1]} {stat_name}: {team2_value_elements[0].text.strip()}")
#         if len(values) == 2:  # We expect exactly two values (one per team)
#             all_stats[stat_name] = values
#         else:
#             print(f"  Skipped due to unexpected format: {values}")

        all_stats[stat_name] = [team1_value_elements[0].text.strip().replace('%', '').replace('.', ''), team2_value_elements[0].text.strip().replace('%', '').replace('.', '')]
       
        # Extract values if present, else set to default "1/1"
#         team1_value = team1_value_elements[0].text.strip().replace('%', '').replace('.', '') if team1_value_elements else "0/1"
#         team2_value = team2_value_elements[0].text.strip().replace('%', '').replace('.', '') if team2_value_elements else "1/10"

#         all_stats[stat_name] = [team1_value, team2_value]
        
    except Exception as e:
        print(f"Error extracting {stat_name}: {e}")

# Create a DataFrame
if all_stats and len(teams) == 2:
    xvy_df = pd.DataFrame.from_dict(all_stats, orient='index', columns=teams).transpose()
#     print("\nExtracted Data:")
#     print(df)

    # Apply transformation to all columns except 'Team'
    for col in xvy_df.columns:  
        xvy_df[col] = xvy_df[col].apply(to_percentage)

    xvy_df = xvy_df.astype('float')
    
    # Creating the new aggregated df
    # eg: 2v4, 1v3, and 3v5 all get aggregated into "+2"
    agg_data = {}
    counts = {}

    for col in xvy_df.columns:
        category = get_advantage(col)
        if category not in agg_data:
            agg_data[category] = xvy_df[col].copy()
            counts[category] = 1
        else:
            agg_data[category] += xvy_df[col]
            counts[category] += 1

    xvy_df = pd.DataFrame({key: agg_data[key] / counts[key] for key in agg_data})

    # Save to CSV
#     df.to_csv("team_stats.csv")
#     print("Data saved to team_stats.csv")
else:
    print("Failed to extract meaningful stat data")
    
df = pd.concat([main_df, eco_df, xvy_df], axis=1)
df = df.reset_index(drop=True)

df.insert(0, "Team", teams)  # Ensure "Team" is always the first column

# Copy stats from row 1 to row 0
df.loc[0, 'Pistol Rounds Won Opp'] = df.loc[1, 'Pistol Rounds Won']
df.loc[0, 'First Kills Opp'] = df.loc[1, 'First Kills']
df.loc[0, 'KAST Opp'] = df.loc[1, 'KAST']
df.loc[0, 'Clutches Opp'] = df.loc[1, 'Clutches']
df.loc[0, 'Eco Opp'] = df.loc[1, 'Eco']
df.loc[0, 'Semi-Eco Opp'] = df.loc[1, 'Semi-Eco']
df.loc[0, 'Half-Buy Opp'] = df.loc[1, 'Half-Buy']
df.loc[0, 'Full-Buy Opp'] = df.loc[1, 'Full-Buy']
df.loc[0, '+1 Opp'] = df.loc[1, '+1']
df.loc[0, '-1 Opp'] = df.loc[1, '-1']
df.loc[0, '0 Opp'] = df.loc[1, '0']
df.loc[0, '+2 Opp'] = df.loc[1, '+2']
df.loc[0, '-2 Opp'] = df.loc[1, '-2']
df.loc[0, '-3 Opp'] = df.loc[1, '-3']
df.loc[0, '+3 Opp'] = df.loc[1, '+3']
# df.loc[0, '+4 Opp'] = df.loc[1, '+4']
# df.loc[0, '-4 Opp'] = df.loc[1, '-4']
try:
    df.loc[0, '+4 Opp'] = df.loc[1, '+4']
except KeyError:
    print("Column '+4' not found, setting default value.")
    df.loc[0, '+4 Opp'] = 1.0  # Default value to avoid error
try:
    df.loc[0, '-4 Opp'] = df.loc[1, '-4']
except KeyError:
    print("Column '-4' not found, setting default value.")
    df.loc[0, '-4 Opp'] = 0.0  # Default value to avoid error
    

# Copy stats from row 0 to row 1
df.loc[1, 'Pistol Rounds Won Opp'] = df.loc[0, 'Pistol Rounds Won']
df.loc[1, 'First Kills Opp'] = df.loc[0, 'First Kills']
df.loc[1, 'KAST Opp'] = df.loc[0, 'KAST']
df.loc[1, 'Clutches Opp'] = df.loc[0, 'Clutches']
df.loc[1, 'Eco Opp'] = df.loc[0, 'Eco']
df.loc[1, 'Semi-Eco Opp'] = df.loc[0, 'Semi-Eco']
df.loc[1, 'Half-Buy Opp'] = df.loc[0, 'Half-Buy']
df.loc[1, 'Full-Buy Opp'] = df.loc[0, 'Full-Buy']
df.loc[1, '+1 Opp'] = df.loc[0, '+1']
df.loc[1, '-1 Opp'] = df.loc[0, '-1']
df.loc[1, '0 Opp'] = df.loc[0, '0']
df.loc[1, '+2 Opp'] = df.loc[0, '+2']
df.loc[1, '-2 Opp'] = df.loc[0, '-2']
df.loc[1, '-3 Opp'] = df.loc[0, '-3']
df.loc[1, '+3 Opp'] = df.loc[0, '+3']
# df.loc[1, '+4 Opp'] = df.loc[0, '+4']
# df.loc[1, '-4 Opp'] = df.loc[0, '-4']
try:
    df.loc[1, '+4 Opp'] = df.loc[0, '+4']
except KeyError:
    print("Column '+4' not found, setting default value.")
    df.loc[1, '+4 Opp'] = 1.0  # Default value to avoid error
    
try:
    df.loc[1, '-4 Opp'] = df.loc[0, '-4']
except KeyError:
    print("Column '-4' not found, setting default value.")
    df.loc[1, '-4 Opp'] = 0.0  # Default value to avoid error

# Map results based on teams
df["Result"] = df["Team"].map(dict(zip(teams, [1 if r == "Winner" else 0 for r in results])))

driver.quit()

Waiting for page to load...
Teams detected: ['Cloud9', 'MIBR']
Results detected: ['Loser', 'Winner']
       Pistol Rounds Won First Kills KAST Clutches
Cloud9                 3          36   74   5 / 38
MIBR                   3          38   75  12 / 45
Column '+4' not found, setting default value.
Column '-4' not found, setting default value.
Column '+4' not found, setting default value.
Column '-4' not found, setting default value.


In [36]:
match_id = "87933" # Choose the Match ID from rib.gg
url = f"https://www.rib.gg/series/{match_id}?tab=team-stats"

In [38]:
df

,Team,Pistol Rounds Won,First Kills,KAST,Clutches,Eco,Semi-Eco,Half-Buy,Full-Buy,+1,...,+1 Opp,-1 Opp,0 Opp,+2 Opp,-2 Opp,-3 Opp,+3 Opp,+4 Opp,-4 Opp,Result
0,Cloud9,3,36,0.74,0.131579,0.0,0.00,0.47,0.55,0.8400,...,0.8575,0.1600,0.6625,0.923333,0.090000,0.0,1.0,1.0,0.0,0
1,MIBR,3,38,0.75,0.266667,0.0,0.25,0.46,0.58,0.8575,...,0.8400,0.1425,0.3375,0.910000,0.076667,0.0,1.0,1.0,0.0,1


In [7]:
# temp_df = stage1_df.copy()

In [39]:
temp_df = pd.concat([temp_df, df], ignore_index=True)

In [40]:
temp_df

,Team,Pistol Rounds Won,First Kills,KAST,Clutches,Eco,Semi-Eco,Half-Buy,Full-Buy,+1,...,+1 Opp,-1 Opp,0 Opp,+2 Opp,-2 Opp,-3 Opp,+3 Opp,+4 Opp,-4 Opp,Result
0,Cloud9,3,31,0.70,0.156250,0.33,0.00,0.57,0.59,0.8925,...,0.7400,0.1075,0.497500,1.000000,0.000000,0.00,1.00,1.0,0.0,1
1,NRG Esports,3,29,0.72,0.111111,0.00,0.25,0.61,0.38,0.7400,...,0.8925,0.2600,0.502500,1.000000,0.000000,0.00,1.00,1.0,0.0,0
2,KRÜ Esports,4,29,0.74,0.172414,0.00,0.50,0.56,0.56,0.6600,...,0.7125,0.3400,0.445000,0.860000,0.000000,0.00,1.00,1.0,0.0,1
3,LOUD,2,25,0.70,0.151515,0.25,0.00,0.44,0.60,0.7125,...,0.6600,0.2875,0.555000,1.000000,0.140000,0.00,1.00,1.0,0.0,0
4,G2 Esports,3,25,0.75,0.227273,0.00,0.25,0.67,0.67,0.8525,...,0.7250,0.1475,0.317500,1.000000,0.033333,0.06,1.00,0.0,0.0,1
5,FURIA,1,18,0.67,0.153846,0.33,0.00,0.45,0.43,0.7250,...,0.8525,0.2750,0.682500,0.966667,0.000000,0.00,0.94,1.0,0.0,0
6,MIBR,4,23,0.76,0.300000,0.00,0.00,0.55,0.64,0.8725,...,0.7225,0.1275,0.193333,1.000000,0.000000,0.00,1.00,0.0,0.0,1
7,Leviatán,0,19,0.67,0.076923,0.00,0.00,0.00,0.60,0.7225,...,0.8725,0.2775,0.806667,1.000000,0.000000,0.00,1.00,1.0,0.0,0
8,2Game Esports,1,23,0.63,0.161290,0.25,0.00,0.25,0.56,0.6775,...,0.8750,0.3225,0.635000,1.000000,0.223333,0.00,1.00,1.0,0.0,0
9,Evil Geniuses,3,22,0.76,0.333333,1.00,0.00,0.60,0.62,0.8750,...,0.6775,0.1250,0.365000,0.776667,0.000000,0.00,0.50,1.0,0.0,1


---

## Saving and Loading the Datasets

In [16]:
kickoff_df = pd.read_csv('Datasets/kickoff.csv')
kickoff_df = kickoff_df.drop(columns=['Unnamed: 0'])

In [90]:
kickoff_df.to_csv('Datasets/kickoff.csv')

In [45]:
masters_df = pd.read_csv('Datasets/kickoff_with_masters.csv')
masters_df = masters_df.drop(columns=['Unnamed: 0'])

In [63]:
masters_df.to_csv('Datasets/kickoff_with_masters.csv')

In [46]:
stage1_df = pd.read_csv('Datasets/stage1.csv')
stage1_df = stage1_df.drop(columns=['Unnamed: 0'])

In [42]:
stage1_df.to_csv('Datasets/stage1.csv')

In [47]:
total_df = pd.concat([masters_df, stage1_df], ignore_index=True)

In [48]:
total_df

,Team,Pistol Rounds Won,First Kills,KAST,Clutches,Eco,Semi-Eco,Half-Buy,Full-Buy,+1,...,+1 Opp,-1 Opp,0 Opp,+2 Opp,-2 Opp,-3 Opp,+3 Opp,+4 Opp,-4 Opp,Result
0,Evil Geniuses,2,23,0.67,0.068966,0.0,0.25,0.33,0.52,0.7200,...,0.8650,0.2800,0.6050,0.916667,0.046667,0.00,1.00,1.0,0.0,0
1,LOUD,2,26,0.71,0.173913,0.0,0.00,0.43,0.68,0.8650,...,0.7200,0.1350,0.3950,0.953333,0.083333,0.00,1.00,1.0,0.0,1
2,MIBR,1,11,0.59,0.040000,0.0,0.00,0.25,0.53,0.7000,...,0.7925,0.3000,0.6750,0.953333,0.166667,0.00,1.00,1.0,0.0,0
3,100 Thieves,3,28,0.82,0.200000,0.0,0.00,0.44,0.76,0.7925,...,0.7000,0.2075,0.3250,0.833333,0.046667,0.00,0.50,0.0,0.0,1
4,2Game Esports,2,20,0.67,0.071429,0.0,0.00,0.38,0.37,0.6550,...,0.9450,0.3450,0.9000,0.850000,0.110000,0.00,1.00,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,MIBR,2,38,0.73,0.157895,0.0,0.25,0.42,0.56,0.8750,...,0.8200,0.1250,0.5100,1.000000,0.056667,0.00,1.00,0.0,0.0,0
80,LOUD,1,23,0.69,0.142857,0.0,0.33,0.71,0.48,0.7350,...,0.7450,0.2650,0.5450,1.000000,0.000000,0.00,0.93,1.0,0.0,0
81,2Game Esports,3,27,0.76,0.178571,0.0,0.14,0.67,0.59,0.7450,...,0.7350,0.2550,0.4550,1.000000,0.000000,0.07,1.00,1.0,0.0,1
82,Leviatán,1,14,0.58,0.037037,0.0,0.14,0.50,0.31,0.7400,...,0.9575,0.2600,0.7375,1.000000,0.000000,0.00,1.00,1.0,0.0,0


In [135]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Team                   120 non-null    object 
 1   Pistol Rounds Won      120 non-null    int64  
 2   First Kills            120 non-null    int64  
 3   KAST                   120 non-null    float64
 4   Clutches               120 non-null    float64
 5   Eco                    120 non-null    float64
 6   Semi-Eco               120 non-null    float64
 7   Half-Buy               120 non-null    float64
 8   Full-Buy               120 non-null    float64
 9   +1                     120 non-null    float64
 10  -1                     120 non-null    float64
 11  0                      120 non-null    float64
 12  +2                     120 non-null    float64
 13  -2                     120 non-null    float64
 14  -3                     120 non-null    float64
 15  +3    